# The Patron
## An example use case of the Digital Twin

In [16]:
from azure.identity import AzureCliCredential
from azure.digitaltwins.core import DigitalTwinsClient

# using yaml instead of 
import yaml
import uuid

# using altair instead of matplotlib for vizuals
import altair as alt
import numpy as np
import pandas as pd


azure_cli = AzureCliCredential()
service_client = DigitalTwinsClient(
    "home-test-twin.api.wcus.digitaltwins.azure.net", azure_cli)
service_client

In [2]:
service_client

Loading the model from a local Json file, I'm defining a model for a `Patron`. That's like a `class` that will be used to to make several instances of a customer. 

In [3]:
patron_model_id = "dtmi:billmanh:patron;1"

In [4]:
# # Delete the model that you don't want. 
# service_client.delete_model(patron_model_id)

# # Create it if you just deleted it.
# patron_model_json = yaml.safe_load(open("../models/patron.json"))
# service_client.create_models([patron_model_json])

Note that the jason must be in a list. This is so that you can deploy several models in a single go.

In [5]:
# Get the Patron model
get_model = service_client.get_model(patron_model_id)
get_model.as_dict()

{'display_name': {'en': 'Patron'},
 'description': {'en': 'As an example, contains all of the properties possible in the DTDL.'},
 'id': 'dtmi:billmanh:patron;1',
 'upload_time': '2020-11-19T02:14:27.21094Z',
 'decommissioned': False}

In my case I use `customer` as an _instance_ of a model (a _twin_). The only thing to add is the information specific to the twin I want to upload. 

In [6]:
digital_twin_id = 'customer-' + str(uuid.uuid4())

customer_json = {
    "$metadata": {
        "$model": patron_model_id
    },
    "satisfaction": 10,
    "totalWaitTime": 10
}

created_twin = service_client.upsert_digital_twin(digital_twin_id, customer_json)

You can see that the process with the API is pretty simple:
* Define a model
* Build a twin from that model
* Give that model some attributes. 

In [7]:
type(created_twin)

dict

In [8]:
created_twin

{'$dtId': 'customer-3cbd5e60-957d-44ff-944f-9adb42a20a52',
 '$etag': 'W/"ea33ae65-f47c-4116-8e3d-1c6fa61777d6"',
 'satisfaction': 10,
 'totalWaitTime': 10,
 '$metadata': {'$model': 'dtmi:billmanh:patron;1',
  'satisfaction': {'lastUpdateTime': '2020-11-19T15:48:13.1057699Z'},
  'totalWaitTime': {'lastUpdateTime': '2020-11-19T15:48:13.1057699Z'}}}

So let's create a bunch of people. Let's pretend we have a single room full of a bunch of people that all have simmilar attributes:
* `satisfaction` varies from person to person.
* `totalWaitTime` is the same at the beginning. 

the item for the `$metadata` is the model that it will be an instance of. The rest of the items in the dict are the `contents` of the model as you defined it. 

In [13]:
def create_new_customer():
    digital_twin_id = 'customer-' + str(uuid.uuid4())
    customer_json = {
        "$metadata": {
            "$model": patron_model_id
        },
        "satisfaction": np.random.randint(5,10),
        "totalWaitTime": 0
    }
    return digital_twin_id,customer_json
    
customer_twin_examples = [create_new_customer() for i in range(40)]
customer_twin_examples[:5]

[('customer-5c454e2f-f70b-4352-b75a-958f1a49beba',
  {'$metadata': {'$model': 'dtmi:billmanh:patron;1'},
   'satisfaction': 7,
   'totalWaitTime': 0}),
 ('customer-26196fee-5ffd-457a-86b7-192a998f3cf2',
  {'$metadata': {'$model': 'dtmi:billmanh:patron;1'},
   'satisfaction': 9,
   'totalWaitTime': 0}),
 ('customer-e6f49d8a-711b-41c3-9db8-c7ece3dbc32c',
  {'$metadata': {'$model': 'dtmi:billmanh:patron;1'},
   'satisfaction': 7,
   'totalWaitTime': 0}),
 ('customer-c87adbfa-1c6e-4ea9-9f03-83e3877ef5fc',
  {'$metadata': {'$model': 'dtmi:billmanh:patron;1'},
   'satisfaction': 8,
   'totalWaitTime': 0}),
 ('customer-21e17d28-76c3-4c04-8df9-396703692a68',
  {'$metadata': {'$model': 'dtmi:billmanh:patron;1'},
   'satisfaction': 8,
   'totalWaitTime': 0})]

In [22]:
customer_df = pd.DataFrame([i[1] for i in customer_twin_examples])

alt.Chart(customer_df).mark_bar().encode(
    alt.X("satisfaction:Q", bin=True),
    alt.Y('count()', title='# customers')
).properties()

alt.Chart(...)

Of course those are just local customer data points. You could imagine that this is information collected at the edge. 

Now let's upload those people to see them in the cloud. Try to imagine that each customer in the store had an app on thier phone that sent this information to the graph. Each `customer` represents a datafeed from IoT telemetry. Generally each device would upload a separate feed of IoT, but we are simulating using a single device (your computer) to upload this data. 

In [24]:
for i in customer_twin_examples:
    service_client.upsert_digital_twin(i[0], i[1])

Now let's query our customers to see how they are feeling. 

In [26]:
query_expression = 'SELECT * FROM digitaltwins'
query_result = service_client.query_twins(query_expression)

In [28]:
for i in query_result:
    pass

In [29]:
i

{'$dtId': 'customer-2e551c55-265a-46e0-a84b-4965bc734e21',
 '$etag': 'W/"8f1be988-70af-4488-a7c9-5403a4b6ea1d"',
 'satisfaction': 9,
 'totalWaitTime': 0,
 '$metadata': {'$model': 'dtmi:billmanh:patron;1',
  'satisfaction': {'lastUpdateTime': '2020-11-19T16:13:44.9882083Z'},
  'totalWaitTime': {'lastUpdateTime': '2020-11-19T16:13:44.9882083Z'}}}

But let's say I've got an application that just reports the overal customer satisfaction from an app.